아래는 ALO 기본 설정 및 라이브러리 설치 코드입니다. 설치 에러가 발생하면 아래 셀을 재실행 하고, 지속적으로 문제가 있을 시 문의바랍니다

In [ ]:
import argparse
import time
import os
os.chdir(os.path.abspath(os.path.join('./alo')))
from src.alo import ALO
alo = ALO(); alo.preset(); pipelines = list(alo.asset_source.keys())
from src.external import external_load_data, external_save_artifacts

## Train workflow 

In [ ]:
# 아래는 Train 시 필요한 라이브러리를 설치하는 코드입니다. library 설치 에러가 발생하면 아래 셀을 재실행 해주세요
external_load_data(pipelines[0], alo.external_path, alo.external_path_permission, alo.control['get_external_data'])
pipeline = pipelines[0]
alo.install_steps(pipeline, alo.control["get_asset_source"])

### 1. Input asset 
##### Input asset의 arguments 수정 및 확인
- 필요한경우 input_args의 항목을 ***input_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# TCR train asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - sampling(2) - train(3))
step = 0 
input_args = alo.user_parameters[pipeline][step]['args'][0].copy()

# 아래 주석을 풀어 input_args를 원하는 값으로 수정합니다. 
# input_args['x_columns'] = ['input_x0','input_x1']
input_args

##### Input asset 실행 

In [ ]:
data = pipe_val = {} # 초기 input asset process 세팅

data_input, config_input = alo.process_asset_step(alo.asset_source[pipeline][step], step, pipeline, data, pipe_val, [input_args])
# data_input: input asset의 결과물입니다. 다음 asset 실행 시 필요합니다. 
# config_input: input asset의 결과 config입니다. 다음 asset실행 시 필요합니다.

# input asset의 결과 dataframe은 data_input['dataframe']으로 확인할 수 있습니다. 
data_input['dataframe'].head(10) 

</br>

### 2. Preprocess asset 
##### Preprocess asset의 args수정 및 확인
- 필요한경우 preprocess_args의 항목을 ***preprocess_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# TCR train asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - sampling(2) - train(3))
step = 1 
preprocess_args = alo.user_parameters[pipeline][step]['args'][0].copy()

# 아래 주석을 풀어 preprocess_args 수정합니다. 
# preprocess_args['handling_missing'] = 'interpolation'
preprocess_args

##### Preprocess asset 실행 

In [ ]:
data_preprocess, config_preprocess = alo.process_asset_step(alo.asset_source[pipeline][step], step, pipeline, data_input, config_input.copy(), [preprocess_args])
# data_preprocess: preprocess asset의 결과물입니다. 다음 asset 실행 시 필요합니다. 
# config_preprocess: preprocess asset의 결과 config입니다. 다음 asset실행 시 필요합니다. 

# preprocess asset의 결과 dataframe은 data_preprocess['dataframe']으로 확인할 수 있습니다. 
data_preprocess['dataframe'].head(10) 

</br>

### 3. Sampling asset 
##### Sampling asset의 args수정 및 확인
- 필요한경우 Sampling_args의 항목을 ***sampling_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# TCR train asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - sampling(2) - train(3))
step = 2 
sampling_args = alo.user_parameters[pipeline][step]['args'][0].copy()

# 아래 주석을 풀어 preprocess_args 수정합니다. 
# sampling_args['sampling_type'] = 'under'
sampling_args

##### Sampling asset 실행 

In [ ]:
data_sampling, config_sampling = alo.process_asset_step(alo.asset_source[pipeline][step], step, pipeline, data_preprocess, config_preprocess.copy(), [sampling_args])
# data_sampling: sampling asset의 결과물입니다. 다음 asset 실행 시 필요합니다. 
# config_sampling: sampling asset의 결과 config입니다. 다음 asset실행 시 필요합니다. 

# sampling asset의 결과 dataframe은 data_sampling['dataframe']으로 확인할 수 있습니다. 
data_sampling['dataframe'].head(10) 

</br>

### 4. train asset 
##### train asset의 args수정 및 확인
- 필요한경우 TCR_args의 항목을 ***TCR_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# TCR train asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - sampling(2) - train(3))
step = 3 
tcr_args = alo.user_parameters[pipeline][step]['args'][0].copy()

# 아래 주석을 풀어 tcr_args를 수정합니다. 
# tcr_args['model_list'] = ['lgb']
tcr_args

##### train asset 실행 

In [ ]:
data_tcr, config_tcr = alo.process_asset_step(alo.asset_source[pipeline][step], step, pipeline, data_sampling, config_sampling.copy(), [tcr_args])
# data_tcr: TCR asset의 결과물입니다. 
# config_tcr: TCR asset의 결과 config입니다. 

# tcr asset의 결과 dataframe은 data_tcr['dataframe']으로 확인할 수 있습니다. 
data_tcr['dataframe'].head(10) 

<br>

## Inference workflow 

In [ ]:
# 아래는 Inference 시 필요한 라이브러리를 설치하는 코드입니다. library 설치 에러가 발생하면 아래 셀을 재실행 해주세요
external_load_data(pipelines[1], alo.external_path, alo.external_path_permission, alo.control['get_external_data'])
pipeline = pipelines[1]
alo.install_steps(pipeline, alo.control["get_asset_source"])

### 1. Input asset 
##### Input asset의 arguments 수정 및 확인
- 필요한경우 input_args의 항목을 ***input_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# TCR inference asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - inference(2))
step = 0 
input_args = alo.user_parameters[pipeline][step]['args'][0].copy()

# 아래 주석을 풀어 input_args를 원하는 값으로 수정합니다. 
# input_args['x_columns'] = ['']
input_args

##### Input asset 실행 

In [ ]:
data = pipe_val = {} # 초기 input asset process 세팅

data_input, config_input = alo.process_asset_step(alo.asset_source[pipeline][step], step, pipeline, data, pipe_val, [input_args])
# data_input: input asset의 결과물입니다. 다음 asset 실행 시 필요합니다. 
# config_input: input asset의 결과 config입니다. 다음 asset실행 시 필요합니다.

# input asset의 결과 dataframe은 data_input['dataframe']으로 확인할 수 있습니다. 
data_input['dataframe'].head(10) 

</br>

### 2. Preprocess asset 
##### Preprocess asset의 args수정 및 확인
- 필요한경우 preprocess_args의 항목을 ***preprocess_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# TCR inference  asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - inference(2))
step = 1 
preprocess_args = alo.user_parameters[pipeline][step]['args'][0].copy()

# 아래 주석을 풀어 preprocess_args 수정합니다. 
# preprocess_args['handling_missing'] = 'interpolation'
preprocess_args

##### Preprocess asset 실행 

In [ ]:
data_preprocess, config_preprocess = alo.process_asset_step(alo.asset_source[pipeline][step], step, pipeline, data_input, config_input.copy(), [preprocess_args])
# data_preprocess: preprocess asset의 결과물입니다. 다음 asset 실행 시 필요합니다. 
# config_preprocess: preprocess asset의 결과 config입니다. 다음 asset실행 시 필요합니다. 

# preprocess asset의 결과 dataframe은 data_preprocess['dataframe']으로 확인할 수 있습니다. 
data_preprocess['dataframe'].head(10) 

<br>

### 3. inference asset 
##### inference asset의 args수정 및 확인
- 필요한경우 TCR_args의 항목을 ***TCR_args[argument명]=value입력*** 을 통해 변경할 수 있습니다.  

In [ ]:
# TCR inference asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - inference(2))
step = 2
tcr_args = alo.user_parameters[pipeline][step]['args'][0].copy()

# 아래 주석을 풀어 tcr_args를 수정합니다. 
# tcr_args['model_type'] = 
tcr_args

##### inference asset 실행 

In [ ]:
data_tcr, config_tcr = alo.process_asset_step(alo.asset_source[pipeline][step], step, pipeline, data_preprocess, config_preprocess.copy(), [tcr_args])
# data_tcr: TCR asset의 결과물입니다. 
# config_tcr: TCR asset의 결과 config입니다. 

# tcr asset의 결과 dataframe은 data_tcr['dataframe']으로 확인할 수 있습니다. 
data_tcr['dataframe'].head(10) 